# _de novo_ RADseq assembly using _pyRAD_

# Modification to start looking at Ostrea data

Begin by executing the command below. This will download an example simulated RADseq data set and unarchive it into your current directory.

In [2]:
pwd

u'/Users/sr320/git-repos/nb-2016/O_lurida'

In [3]:
cd /Volumes/web/halfshell/working-directory


/Volumes/web/halfshell/working-directory


In [ ]:
mkdir 16-05-27

In [4]:
cd 16-05-27

/Volumes/web/halfshell/working-directory/16-05-27


In [ ]:
ls | head

------------   

The params file lists on each line one parameter followed by a __##__ mark, after which any comments can be left. In the comments section there is a description of the parameter and in parentheses the step of the analysis affected by the parameter. Lines 1-12 are required, the remaining lines are optional. The params.txt file is further described in the general tutorial.

### evolving params file

In [5]:
%%bash
cat params.txt

==** parameter inputs for pyRAD version 3.0.64  **======================== affected step ==
./                        ## 1. Working directory                                 (all)
./*.fq.gz              ## 2. Loc. of non-demultiplexed files (if not line 18)  (s1)
./*.barcodes              ## 3. Loc. of barcode file (if not line 18)             (s1)
/Applications/bioinfo/vsearch-1.11.1-osx-x86_64/bin/vsearch                   ## 4. command (or path) to call vsearch (or usearch)    (s3,s6)
/Applications/bioinfo/muscle3.8.31_i86darwin64                    ## 5. command (or path) to call muscle                  (s3,s7)
CWGC                      ## 6. Restriction overhang (e.g., C|TGCAG -> TGCAG)     (s1,s2)
6                         ## 7. N processors (parallel)                           (all)
5                         ## 8. Mindepth: min coverage for a cluster              (s4,s5)
4                         ## 9. NQual: max # sites with qual < 20 (or see line 20)(s2)
.88                   

#### To change parameters you can edit params.txt in any text editor. Here to automate things I use the script below.

--------------   

__Let's take a look at what the raw data look like.__

Your input data will be in fastQ format, usually ending in .fq or .fastq. Your data could be split among multiple files, or all within a single file (de-multiplexing goes much faster if they happen to be split into multiple files). The file/s may be compressed with gzip so that they have a .gz ending, but they do not need to be. The location of these files should be entered on line 2 of the params file. Below are the first three reads in the example file.

## Sample Description

<img src="http://eagle.fish.washington.edu/cnidarian/skitch/Genotype_by_sequencing_November_2015_·_RobertsLab_project-olympia_oyster-genomic_Wiki_🔊_1CEB70ED.png" alt="Genotype_by_sequencing_November_2015_·_RobertsLab_project-olympia_oyster-genomic_Wiki_🔊_1CEB70ED.png"/>

In [ ]:
!gunzip *.gz

----------------   

## Step 1: de-multiplexing ##

In [ ]:
already done by bgi

### Step 2: quality filtering

In [6]:
%%bash
pyRAD -p params.txt -s 2

	/Volumes/web/halfshell/working-directory/16-05-27/edits/1NF_13A_1 already in edits/
	/Volumes/web/halfshell/working-directory/16-05-27/edits/1NF_29A_1 already in edits/
	/Volumes/web/halfshell/working-directory/16-05-27/edits/1SN_30A_1 already in edits/
	/Volumes/web/halfshell/working-directory/16-05-27/edits/1HL_28A_1 already in edits/
	/Volumes/web/halfshell/working-directory/16-05-27/edits/1SN_23A_1 already in edits/
	/Volumes/web/halfshell/working-directory/16-05-27/edits/1NF_4A_1 already in edits/




     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------

	sorted .fastq from /Volumes/web/halfshell/working-directory/16-05-17/fastq/ being used
	step 2: editing raw reads 
	..........................................................................................

In [8]:
%%bash
ls edits/ |head

1HL_10A_1.edit
1HL_11A_1.edit
1HL_12A_1.edit
1HL_13A_1.edit
1HL_14A_1.edit
1HL_15A_1.edit
1HL_16A_1.edit
1HL_17A_1.edit
1HL_19A_1.edit
1HL_1A_1.edit


The filtered data are written in fasta format (quality scores removed) into a new directory called edits/. Below I show a preview of the file which you can view most easily using the `less` command (I use `head` here to make it fit in the text window better).

In [9]:
%%bash
head -n 10 edits/1HL_10A_1.edit | cut -c 1-80

>1HL_10A_1_0_r1
CTGCAAATTGTGTGTCAAGAAAAAAATTGCATACGAAAAATTTACTTGCCACAAGCTTGCCGAAAACGTGTGCCACAAAT
>1HL_10A_1_1_r1
CTGCGTCAGAAACTGTTCCAAGCAGAAACGCGTATCTGACGTCACCCATGTTTTCCCGCGTGCGAGTTAGCCACGTCTCA
>1HL_10A_1_2_r1
CTGCAATTTTTTTCATTTGATAAGTTTTCATCACAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAG
>1HL_10A_1_3_r1
CTGCAATTTTTTTCATTTGATAAGTTTTCATCACAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAG
>1HL_10A_1_4_r1
CTGCTATTCATAACAAACATGTGTTCAGCGCTCAGTTCATTGACTGACTAGGGTTAGGGTACACAAGCATGAATATCAAT


### Step 3: clustering within-samples

Step 3 de-replicates and then clusters reads within each sample by the set clustering threshold and writes the clusters to new files in a directory called clust.xx

In [10]:
%%bash
pyRAD -p params.txt -s 3



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	de-replicating files for clustering...

	step 3: within-sample clustering of 96 samples at 
	        '.88' similarity. Running 6 parallel jobs
	 	with up to 6 threads per job. If needed, 
		adjust to avoid CPU and MEM limits

	sample 1NF_29A_1 finished, 206813 loci
	sample 1HL_28A_1 finished, 221001 loci
	sample 1SN_23A_1 finished, 237798 loci
	sample 1NF_4A_1 finished, 227834 loci
	sample 1SN_30A_1 finished, 243545 loci
	sample 1NF_13A_1 finished, 252424 loci
	sample 1HL_17A_1 finished, 209054 loci
	sample 1HL_23A_1 finished, 205395 loci
	sample 1HL_19A_1 finished, 225740 loci
	sample 1NF_31A_1 finished, 223265 loci
	sample 1SN_14A_1 finished, 222659 loci
	sample 1SN_21A_1 finished, 253840 loci
	sample 1HL_11A_1 finished, 246951 loci
	sample 1NF_2A_1 finished, 211397 loci
	sample 1SN_12A

Once again, I recommend you use the unix command 'less' to look at the clustS files. These contain each cluster separated by "//". For the first few clusters below you can see that there is one or two alleles in the cluster and one or a few reads that contained a (simulated) sequencing error. 

In [ ]:
%%bash
less clust.85/1A0.clustS.gz | head -n 26 | cut -c 1-80

---------------


The stats output tells you how many clusters were found, and their mean depth of coverage. It also tells you how many pass  your minimum depth setting. You can use this information to decide if you wish to increase or decrease the mindepth before it is applied for making consensus base calls in steps 4 & 5.

In [ ]:
%%bash
head -n 40 stats/s3.clusters.txt

### Steps 4 & 5: Call consensus sequences

#### Step 4 jointly infers the error-rate and heterozygosity across samples.

In [11]:
%%bash
pyRAD -p params.txt -s 4



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 4: estimating error rate and heterozygosity
	................................................................................................

In [ ]:
%%bash
less stats/Pi_E_estimate.txt

#### Step 5 calls consensus sequences using the parameters inferred above, and filters for paralogs.

In [12]:
%%bash
pyRAD -p params.txt -s 5



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 5: creating consensus seqs for 96 samples, using H=0.01135 E=0.00214
	................................................................................................

#### The stats output for step 5

In [ ]:
%%bash
less stats/s5.consens.txt

### Step 6: Cluster across samples

Step 6 clusters consensus sequences across samples. It will print its progress to the screen. This uses 6 threads by default. If you enter 0 for param 37 it will use all available processors. 

In [13]:
%%bash
pyRAD -p params.txt -s 6 

vsearch v1.11.1_osx_x86_64, 32.0GB RAM, 8 cores
https://github.com/torognes/vsearch


	finished clustering




     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------


	step 6: clustering across 96 samples at '.88' similarity 

Reading file /Volumes/web/halfshell/working-directory/16-05-27/clust.88/cat.haplos_ 100%
630364266 nt in 6626937 seqs, min 32, max 148, avg 95
Counting unique k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 330947 Size min 1, max 245, avg 20.0
Singletons: 84337, 1.3% of seqs, 25.5% of clusters


## Step 7: Assemble final data sets

The final step is to output data only for the loci that you want to have included in your data set. This filters once again for potential paralogs or highly repetitive regions, and includes options to minimize the amount of missing data in the output. 

In [14]:
%%bash
pyRAD -p params.txt -s 7

	ingroup 1HL_10A_1,1HL_11A_1,1HL_12A_1,1HL_13A_1,1HL_14A_1,1HL_15A_1,1HL_16A_1,1HL_17A_1,1HL_19A_1,1HL_1A_1,1HL_20A_1,1HL_21A_1,1HL_22A_1,1HL_23A_1,1HL_24A_1,1HL_25A_1,1HL_26A_1,1HL_27A_1,1HL_28A_1,1HL_29A_1,1HL_2A_1,1HL_31A_1,1HL_33A_1,1HL_34A_1,1HL_35A_1,1HL_3A_1,1HL_4A_1,1HL_5A_1,1HL_6A_1,1HL_7A_1,1HL_8A_1,1HL_9A_1,1NF_10A_1,1NF_11A_1,1NF_12A_1,1NF_13A_1,1NF_14A_1,1NF_15A_1,1NF_16A_1,1NF_17A_1,1NF_18A_1,1NF_19A_1,1NF_1A_1,1NF_20A_1,1NF_21A_1,1NF_22A_1,1NF_23A_1,1NF_24A_1,1NF_25A_1,1NF_26A_1,1NF_27A_1,1NF_28A_1,1NF_29A_1,1NF_2A_1,1NF_30A_1,1NF_31A_1,1NF_32A_1,1NF_33A_1,1NF_4A_1,1NF_5A_1,1NF_6A_1,1NF_7A_1,1NF_8A_1,1NF_9A_1,1SN_10A_1,1SN_11A_1,1SN_12A_1,1SN_13A_1,1SN_14A_1,1SN_15A_1,1SN_16A_1,1SN_17A_1,1SN_18A_1,1SN_19A_1,1SN_1A_1,1SN_20A_1,1SN_21A_1,1SN_22A_1,1SN_23A_1,1SN_24A_1,1SN_25A_1,1SN_26A_1,1SN_27A_1,1SN_28A_1,1SN_29A_1,1SN_2A_1,1SN_30A_1,1SN_31A_1,1SN_32A_1,1SN_3A_1,1SN_4A_1,1SN_5A_1,1SN_6A_1,1SN_7A_1,1SN_8A_1,1SN_9A_1
	addon 
	exclude 
	
	final stats written to:
	 /Volumes/w



     ------------------------------------------------------------
      pyRAD : RADseq for phylogenetics & introgression analyses
     ------------------------------------------------------------

......

### Final stats output

In [ ]:
%%bash
less stats/c85m4p3.stats

---------------  

## Output formats ##

We created 11 output files from our analysis. The standard two (.loci and .excluded_loci), as well as the 9 additional ones listed in the params file. These are all shown below.

In [ ]:
%%bash 
ls outfiles/

### Loci format  
The ".loci" file contains each locus listed in a fasta-like format that also shows which sites are variable below each locus. Autapomorphies are listed as '-' and shared SNPs as '*'. This is a custom format that is human readable and also used as input to perform D-statistic tests in pyRAD. This is the easiest way to visualize your results. I recommend viewing the file with the command `less`. Below I use a head and cut to make it easy to view in this window.

In [ ]:
%%bash 
head -n 39 outfiles/gbs-001.loci | cut -c 1-75

### PHY format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.phy | cut -c 1-85

### NEX format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.nex | cut -c 1-85

### Alleles format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.alleles| cut -c 1-85

### STRUCTURE (.str) format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.str | cut -c 1-20

### GENO (.geno) format (used in _Admixture_)

In [ ]:
%%bash 
head -n 40 outfiles/c85m4p3.geno 

### SNPs format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.snps | cut -c 1-85

### UNLINKED_SNPs format

In [ ]:
%%bash 
head -n 50 outfiles/c85m4p3.unlinked_snps | cut -c 1-85

## OTHER FORMATS  

You may also produce some more complicated formatting options that involve pooling individuals into groups or populations. This can be done for the "treemix" and "migrate" outputs, which are formatted for input into the programs _TreeMix_ and _migrate-n_, respectively. Grouping individuals into populations is done with the final lines of the params file as shown below, and similar to the assignment of individuals into clades for hierarchical clustering (see full tutorial). 

Each line designates a group, and has three arguments that are separated by space or tab. The first is the group name, the second is the minimum number of individuals that must have data in that group for a locus to be included in the output, and the third is a list of the members of that group. Lists of taxa can include comma-separated names and wildcard selectors, like below. Example:


In [ ]:
%%bash 
## append group designations to the params file
echo "pop1 4 1A0,1B0,1C0,1D0 " >> params.txt
echo "pop2 4 2E0,2F0,2G0,2H0 " >> params.txt
echo "pop3 4 3* " >> params.txt

## view params file
cat params.txt

## Creating population output files  
Now if we run _pyRAD_ with the 'm' (migrate) or 't' (treemix) output options, it will create their output files. 

In [ ]:
%%bash 
pyRAD -p params.txt -s 7

## TREEMIX format

In [ ]:
%%bash 
less outfiles/c85m4p3.treemix.gz | head -n 30

## MIGRATE-n FORMAT

In [ ]:
%%bash 
head -n 40 outfiles/c85m4p3.migrate | cut -c 1-85